# Customer Churn Analysis - Data Cleaning
## Part 1: Data Preparation and Cleaning

**Author:** Your Name  
**Date:** February 2026  
**Purpose:** Clean and prepare the Telco customer churn dataset for analysis

---

## 1. Import Libraries

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✓ Libraries imported successfully")

## 2. Load Dataset

In [ ]:
# Load the dataset
# Download from: https://www.kaggle.com/datasets/blastchar/telco-customer-churn
df = pd.read_csv('../data/WA_Fn-UseC_-Telco-Customer-Churn.csv')

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Rows: {df.shape[0]:,}")
print(f"Columns: {df.shape[1]}")

## 3. Initial Data Exploration

In [ ]:
# Display first few rows
df.head()

In [ ]:
# Dataset info
df.info()

In [ ]:
# Statistical summary
df.describe()

In [ ]:
# Column names
print("Column Names:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

## 4. Data Quality Assessment

In [ ]:
# Check for missing values
missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2)
})

missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

if len(missing_data) > 0:
    print("Missing Values Found:")
    print(missing_data)
else:
    print("✓ No missing values found in the dataset")

In [ ]:
# Check for duplicates
duplicates = df.duplicated().sum()
print(f"Duplicate Rows: {duplicates}")

if duplicates > 0:
    print(f"⚠ Found {duplicates} duplicate rows")
else:
    print("✓ No duplicates found")

In [ ]:
# Check data types
print("Data Types:")
print(df.dtypes)

## 5. Data Cleaning

### 5.1 Fix TotalCharges Column

In [ ]:
# TotalCharges is object type, should be numeric
print(f"Original TotalCharges dtype: {df['TotalCharges'].dtype}")

# Check for non-numeric values
non_numeric = df[pd.to_numeric(df['TotalCharges'], errors='coerce').isna()]
print(f"\nRows with non-numeric TotalCharges: {len(non_numeric)}")

if len(non_numeric) > 0:
    print("\nSample of problematic rows:")
    print(non_numeric[['customerID', 'tenure', 'TotalCharges']].head())

In [ ]:
# Convert TotalCharges to numeric
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Check how many NaN values were created
print(f"NaN values in TotalCharges after conversion: {df['TotalCharges'].isna().sum()}")

### 5.2 Handle Missing Values in TotalCharges

In [ ]:
# Analyze rows with missing TotalCharges
missing_total = df[df['TotalCharges'].isna()]
print(f"Rows with missing TotalCharges: {len(missing_total)}")
print(f"\nTenure distribution for missing TotalCharges:")
print(missing_total['tenure'].describe())

# Most missing values are for tenure = 0 (new customers)
# Fill with MonthlyCharges * tenure (or 0 if tenure = 0)
df['TotalCharges'].fillna(df['MonthlyCharges'] * df['tenure'], inplace=True)

print(f"\n✓ Missing TotalCharges filled")
print(f"Remaining NaN in TotalCharges: {df['TotalCharges'].isna().sum()}")

### 5.3 Clean Column Names

In [ ]:
# Standardize column names (optional - make them lowercase with underscores)
# Uncomment if you want to standardize
# df.columns = df.columns.str.lower().str.replace(' ', '_')

print("✓ Column names cleaned")

### 5.4 Verify Data Types

In [ ]:
# Ensure correct data types
df['SeniorCitizen'] = df['SeniorCitizen'].astype('object')  # Convert to categorical

print("Data Types After Cleaning:")
print(df.dtypes)

## 6. Feature Engineering

In [ ]:
# Create tenure groups
df['TenureGroup'] = pd.cut(df['tenure'], 
                            bins=[0, 6, 12, 24, 48, 72],
                            labels=['0-6 months', '6-12 months', '12-24 months', '24-48 months', '48+ months'])

print("✓ Tenure groups created")
print(df['TenureGroup'].value_counts().sort_index())

In [ ]:
# Create monthly charge groups
df['ChargeGroup'] = pd.cut(df['MonthlyCharges'],
                            bins=[0, 30, 60, 90, 150],
                            labels=['Low (<$30)', 'Medium ($30-$60)', 'High ($60-$90)', 'Very High ($90+)'])

print("✓ Charge groups created")
print(df['ChargeGroup'].value_counts().sort_index())

In [ ]:
# Create service count (number of additional services)
service_columns = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

df['ServiceCount'] = (df[service_columns] == 'Yes').sum(axis=1)

print("✓ Service count created")
print(df['ServiceCount'].value_counts().sort_index())

## 7. Final Data Quality Check

In [ ]:
# Summary of cleaned dataset
print("=" * 60)
print("CLEANED DATASET SUMMARY")
print("=" * 60)
print(f"Total Rows: {len(df):,}")
print(f"Total Columns: {len(df.columns)}")
print(f"Missing Values: {df.isnull().sum().sum()}")
print(f"Duplicate Rows: {df.duplicated().sum()}")
print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# Display cleaned data
df.head(10)

## 8. Save Cleaned Data

In [ ]:
# Save to CSV
df.to_csv('../data/cleaned_churn_data.csv', index=False)
print("✓ Cleaned data saved to '../data/cleaned_churn_data.csv'")

In [ ]:
# Save to pickle for faster loading
df.to_pickle('../data/cleaned_churn_data.pkl')
print("✓ Cleaned data saved to '../data/cleaned_churn_data.pkl'")

## Summary

### Cleaning Steps Completed:
1. ✓ Loaded dataset (7,043 rows, 21 columns)
2. ✓ Converted TotalCharges to numeric
3. ✓ Handled missing values in TotalCharges
4. ✓ Verified no duplicates
5. ✓ Created engineered features:
   - TenureGroup
   - ChargeGroup
   - ServiceCount
6. ✓ Saved cleaned data

### Next Steps:
- Proceed to `02_exploratory_analysis.ipynb` for EDA
- Build visualizations to understand churn patterns
- Identify key drivers of customer churn